### OJO 👁️👁️
con la base de CHATGPT >>
 comparar metodología con el curso

In [ ]:
import pandas as pd
import numpy as np

# 1. Carga de datos
# Supongamos que ya tienes un DataFrame de Airbnb con columnas:
#   rooms, bathrooms, district, price
df = pd.read_csv('airbnb_listings.csv')

# 2. Preprocesado básico
# 2.1. eliminar filas con nulos
df = df.dropna(subset=['rooms','bathrooms','district','price'])

# 2.2. codificar distrito (one-hot)
df = pd.get_dummies(df, columns=['district'], drop_first=True)

# 2.3. separar características y target
X = df.drop('price', axis=1)
y = df['price']

# 3. División train / validation / test
from sklearn.model_selection import train_test_split

# primero train+val vs test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# luego train vs val
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42)
# => 60% train, 20% val, 20% test

# 4. Definición de modelos a comparar
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree':    DecisionTreeRegressor(random_state=42),
    'RandomForest':    RandomForestRegressor(n_estimators=100, random_state=42),
    'SVR':             SVR(),
}

# 5. Entrenamiento y evaluación en validación
from sklearn.metrics import mean_absolute_error, mean_squared_error

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    results.append({'model': name, 'MAE': mae, 'RMSE': rmse})

results_df = pd.DataFrame(results).sort_values('MAE')
print(results_df)


In [ ]:
best_name = results_df.iloc[0]['model']
best_model = models[best_name]

# entrenar de nuevo con todo train+val
best_model.fit(X_trainval, y_trainval)
y_test_pred = best_model.predict(X_test)

mae_test  = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Mejor modelo: {best_name}")
print(f"Test MAE: {mae_test:.2f}, Test RMSE: {rmse_test:.2f}")
